In [1]:
import copy
import os
import time
import logging
import torch
import torch.optim as optim
from dataset import OGBNDataset
from ogb.nodeproppred import Evaluator
from utils.logger import Logger, set_logging
from utils.cluster import partition_graph
from utils.loops import (
    train,
    multi_evaluate)
from gcn import GCN
import warnings

warnings.filterwarnings('ignore')

In [2]:
dataset = OGBNDataset(dataset_name='ogbn-proteins', aggr='mean')

dataset\init_node_features_mean.pt exists


# Build argument for evaluation

In [3]:
class Args:
    def __init__(self, num_layers=5, cluster_type='random', overlap_ratio=0.5, edge_drop=0.5, intervals=1, cluster_number=20, clusters_path=r'dataset/leiden_clusters_80_100_weight_final.csv', save=r'log/EXP_0005lr250-leiden_species_Overlap_05-Aggr_mean-Epochs_1000-Intervals_1-Layers_5-Embed_256-Dropout_0.5-Edge_drop_0.5', model_path='leiden_species-train_20-test_20-num_evals_1_valid_best.pth'):
        self.num_layers = num_layers
        self.cluster_type = cluster_type
        self.overlap_ratio = overlap_ratio
        self.edge_drop = edge_drop
        self.intervals = intervals
        self.cluster_number = cluster_number
        self.clusters_path = clusters_path
        self.dataset = 'ogbn-proteins'
        self.aggr = 'mean'
        self.nruns = 1
        self.use_cpu = False
        self.device = 0
        self.epochs = 1000
        self.num_evals = 1
        self.lr = 0.005
        self.dropout = 0.5
        self.hidden_channels = 256
        self.in_channels = 8
        self.num_tasks = 112
        self.model_save_path = 'model_ckpt'
        self.save = save
        self.model_load_path = model_path
        self.pretrain = os.path.join(save, self.model_save_path, self.model_load_path)

In [4]:
leiden_args = Args()
random_args = Args(edge_drop=0.0, save=r'log/EXP_0005lr250-random-Aggr_mean-Epochs_1000-Intervals_1-Layers_5-Embed_256-Dropout_0.5-Edge_drop_0.0', model_path=r'random-train_20-test_20-num_evals_1_valid_best.pth')

# Evaluation function

In [5]:
def demo_evaluation(args, dataset):
    # ROC-AUC evaluator
    evaluator = Evaluator('ogbn-proteins')

    # Model
    model = GCN(args)
    pretrain = torch.load(args.pretrain)['model_state_dict']
    model.load_state_dict(pretrain)
    model = model.to(args.device)

    data_evals = []
    for i in range(args.num_evals):
        mini_batches = partition_graph(dataset, args)
        data = dataset.generate_sub_graphs(mini_batches, cluster_number=args.cluster_number)
        data_evals.append(data)

    result = multi_evaluate(data_evals, dataset, model, evaluator, args.device)

    print(f"\033[92mTest ROC-AUC: {result['test']['rocauc']}\033[0m")

# Random result

In [6]:
for i in range(5):
    demo_evaluation(random_args, dataset)

Evaluation process: 100%|██████████| 20/20 [00:02<00:00,  9.57it/s]


Test ROC-AUC: 0.78313107357236


Evaluation process: 100%|██████████| 20/20 [00:00<00:00, 28.10it/s]


Test ROC-AUC: 0.7833654667844053


Evaluation process: 100%|██████████| 20/20 [00:00<00:00, 25.46it/s]


Test ROC-AUC: 0.7802637005624123


Evaluation process: 100%|██████████| 20/20 [00:00<00:00, 26.51it/s]


Test ROC-AUC: 0.7821013979245649


Evaluation process: 100%|██████████| 20/20 [00:00<00:00, 25.99it/s]


Test ROC-AUC: 0.7817902572647808


# Leiden result

In [7]:
for i in range(5):
    demo_evaluation(leiden_args, dataset)

Evaluation process: 100%|██████████| 20/20 [00:00<00:00, 29.27it/s]


Test ROC-AUC: 0.7948158326072631


Evaluation process: 100%|██████████| 20/20 [00:00<00:00, 27.72it/s]


Test ROC-AUC: 0.7948684042084887


Evaluation process: 100%|██████████| 20/20 [00:00<00:00, 25.53it/s]


Test ROC-AUC: 0.7945156127991814


Evaluation process: 100%|██████████| 20/20 [00:00<00:00, 25.69it/s]


Test ROC-AUC: 0.7934098247717125


Evaluation process: 100%|██████████| 20/20 [00:00<00:00, 24.91it/s]


Test ROC-AUC: 0.7961941584774133


# Leiden and random with more evaluation

### Random

In [ ]:
random_args.num_evals = 5
for i in range(5):
    demo_evaluation(random_args, dataset)

### Leiden

In [ ]:
leiden_args.num_evals = 5
for i in range(5):
    demo_evaluation(leiden_args, dataset)